In [45]:
import pandas as pd
import numpy as np
from glob import glob
import pymongo
import json

In [2]:
files = [i.split('\\')[-1] for i in glob('./rawDataset/*')]
files.remove('zips')

In [135]:
pds = []
for file in files:
    df = pd.read_csv(
        f'./rawDataset/{file}',
        names=[
            'ProID',
            'Acc#',
            'ModSite',
            'PTM',
            'EvdId',
            'Seq'
        ],
        header=None,
        sep='\t'
    )
    df = df.dropna(how='any')
    pds.append(df)

In [136]:
giant_df = pd.concat(pds, ignore_index=True)

In [137]:
print(giant_df)

               ProID    Acc#  ModSite          PTM              EvdId  \
0         ACTO_ACACA  P18281        1  Acetylation            2376577   
1        14331_ARATH  P42643        2  Acetylation           22223895   
2        14334_ARATH  P46077        2  Acetylation           22223895   
3         ADH1_ARATH  P06525        2  Acetylation           22223895   
4        AB36G_ARATH  Q9XIE2        1  Acetylation           22223895   
...              ...     ...      ...          ...                ...   
2232877   PBL2_ARATH  O49839      254   UMPylation  30948526;26355215   
2232878   RIPK_ARATH  Q9ZUF4      252   UMPylation           22504181   
2232879   BIK1_ARATH  O48814      236   UMPylation           22504181   
2232880   PBL2_ARATH  O49839      253   UMPylation  30948526;26355215   
2232881   RIPK_ARATH  Q9ZUF4      251   UMPylation           22504181   

                           Seq  
0        ----------MNPELQSAIGQ  
1        ---------MATPGASSARDE  
2        ---------MAAPPA

In [138]:
# now we get all unique protein identifiers and proces over them
print(len(giant_df["ProID"].unique()))
print(len(giant_df["Acc#"].unique()))
# Clearly we need to find the accession numbers of the missing ones - and by the looks of it, a lot of them

267611
282886


In [231]:
copy_df = giant_df.copy(deep=True)

copy_df.rename(columns={'ProID': 'Protein Identifier', 'Acc#': 'Accession Number'}, inplace=True)

unique_proteins = list(copy_df[['Protein Identifier', 'Accession Number']].itertuples(index=False, name=None))

copy_df.set_index('Protein Identifier', inplace=True)

database_df = pd.DataFrame(unique_proteins, columns=['Protein Identifier', 'Accession Number'])

unique_proteins = set(t[0] for t in unique_proteins)

# Create a NumPy array from the unique values
unique_proteins = np.array(list(unique_proteins))

In [232]:
database_df['PTMs'] = ''
database_df.set_index('Protein Identifier', inplace=True)
database_df

,Accession Number,PTMs
Protein Identifier,,
ACTO_ACACA,P18281,
14331_ARATH,P42643,
14334_ARATH,P46077,
ADH1_ARATH,P06525,
AB36G_ARATH,Q9XIE2,
...,...,...
PBL2_ARATH,O49839,
RIPK_ARATH,Q9ZUF4,
BIK1_ARATH,O48814,


In [212]:
grouped_modifications = (
    copy_df.groupby(copy_df.index)
    .agg(lambda x: list(x.to_numpy()))
)

In [213]:
grouped_modifications.iloc[0]

Accession Number                             [P32234, P32234, P32234]
ModSite                                                [295, 307, 44]
PTM                 [Phosphorylation, Phosphorylation, Phosphoryla...
EvdId                                  [22817900, 30478224, 25749252]
Seq                 [MWEYLRLQRIYTKPKGQLPDY, KPKGQLPDYNSPVVLHNERTS,...
Name: 128UP_DROME, dtype: object

In [233]:
grouped_modifications['ModSite'] = grouped_modifications['ModSite'].map(lambda x: list(map(int, x)))
grouped_modifications.iloc[0]

Accession Number                             [P32234, P32234, P32234]
ModSite                                                [295, 307, 44]
PTM                 [Phosphorylation, Phosphorylation, Phosphoryla...
EvdId                                  [22817900, 30478224, 25749252]
Seq                 [MWEYLRLQRIYTKPKGQLPDY, KPKGQLPDYNSPVVLHNERTS,...
Name: 128UP_DROME, dtype: object

,Accession Number,PTMs
Protein Identifier,,
ACTO_ACACA,P18281,
14331_ARATH,P42643,
14334_ARATH,P46077,
ADH1_ARATH,P06525,
AB36G_ARATH,Q9XIE2,
...,...,...
ZY11A_HUMAN,Q6WRX3-2,
ZY11B_HUMAN,Q9C0D3-1,
ZY11B_HUMAN,Q9C0D3-2,


In [253]:
import json

for i, protein in enumerate(unique_proteins):
    values = list(
        zip(*
            grouped_modifications.loc[protein, ['ModSite', 'PTM', 'EvdId']].values.tolist()
        )
    )
    database_df.loc[protein, 'PTMs'] = json.dumps(values)
    if i%1000 == 0: print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [263]:
def to_json(row):
    row['PTMs'] = json.loads(row['PTMs'])
    return row

database_df = database_df.apply(to_json, axis=1)

In [265]:
database_df.to_csv('database_df_altered.csv')

In [157]:

for protein in unique_proteins:
    print(database_df.loc[protein])
    print()
    print(copy_df.loc[protein])
    break
    # values = copy_df.loc[protein][['PTM', 'ModSite', 'EvdId']].values
    # database_df.loc[database_df['Protein Identifier'] == protein, 'PTMs'] = values

                   Accession Number PTMs
Protein Identifier                      
K7UKL7_MAIZE                 K7UKL7     
K7UKL7_MAIZE                 K7UKL7     
K7UKL7_MAIZE                 K7UKL7     
K7UKL7_MAIZE                 K7UKL7     

                   Accession Number  ModSite              PTM  \
Protein Identifier                                              
K7UKL7_MAIZE                 K7UKL7      312  Phosphorylation   
K7UKL7_MAIZE                 K7UKL7      313  Phosphorylation   
K7UKL7_MAIZE                 K7UKL7      314  Phosphorylation   
K7UKL7_MAIZE                 K7UKL7      565  Phosphorylation   

                                EvdId                    Seq  
Protein Identifier                                            
K7UKL7_MAIZE                 23933881  KRGEKSKGEYSSSGIQEAERN  
K7UKL7_MAIZE                 23933881  RGEKSKGEYSSSGIQEAERNK  
K7UKL7_MAIZE                 23933881  GEKSKGEYSSSGIQEAERNKL  
K7UKL7_MAIZE        22787273;23933881  WVQSVVRE

In [130]:
print(database_df.iloc[0].PTMs)

[["Acetylation", "1", "2376577", "----------MNPELQSAIGQ"], ["Methylation", "35", "2376577", "APQIENVTVKKVDRSSFLEEV"], ["Methylation", "72", "2376577", "PAIPEDVHVKKVDRGAFLSEI"]]


In [266]:
len(database_df)

283307